In [2]:
import tokenizer
import pickle
import pandas as pd
import numpy as np
import glob
import os
import string
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.util import ngrams
import json
import sparse_matrix_functions
import scipy.sparse as scs

In [3]:
path = '/Users/ale/Dropbox (Yadlin Family)/galvanize/capstone/*.txt'

In [4]:
with open("gene_dictionary_final.pickle", "rb") as dict_gene:
        gene_dict = pickle.load(dict_gene)

In [5]:
gene_keys = set(gene_dict.keys())
gene_values = set(gene_dict.values())

In [6]:
gene_dict['her2']

'#erbb2#'

In [604]:
with open("drug_dictionary_final.pickle", "rb") as dict_drug:
        drug_dict = pickle.load(dict_drug)

In [605]:
drug_keys = set(drug_dict.keys())
drug_values = set(drug_dict.values())

In [606]:
drug_dict['trastuzumab']

'#trastuzumab#'

In [10]:
with open("greek_alphabet.pickle", "rb") as dict_greek:
        greek_dict = pickle.load(dict_greek)

In [11]:
import data_frame_creator

In [12]:
test=data_frame_creator.create_data_frame(path, short_list=True)

In [13]:
test.head()

,files,sentences,#abcg2#,#acadl#,#ackr3#,#acpp#,#ado#,#afuresertib#,#agfg1#,#aicda#,...,#voxtalisib#,#wdtc1#,#wnt5a#,#xs#,#zbp1#,#zbtb8os#,#zfyve9#,#znf746#,resist,sensit
0,guidelines.txt,Something that moves progressivley First thing...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,test.txt,13 Deregulation of the EGFR PI3K PTEN Akt mTO...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,test.txt,ABSTRACT The EGFR PI3K PTEN Akt mTORC1 GSK-3 p...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,test.txt,The expression of this pathway is frequently a...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,test.txt,"In some breast cancer cases, mutations at cert...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
test.shape

(1428, 408)

In [15]:
doc_names, orig_sentences = data_frame_creator.create_corpus()

In [16]:
#orig_sentences

In [17]:
data_frame_creator.process_data(orig_sentences)

Data has been processed and saved


[<2893x267 sparse matrix of type '<class 'numpy.int64'>'
 	with 12901 stored elements in Compressed Sparse Row format>,
 ['#akt1#',
  '#ar#',
  '#brca1#',
  '#brca2#',
  '#carbendazim#',
  '#celp#',
  '#cic#',
  '#doxorubicin#',
  '#egfr#',
  '#erbb2#',
  '#esr1#',
  '#everolimus#',
  '#lapatinib#',
  '#letrozole#',
  '#mice#',
  '#mtor#',
  '#nf1#',
  '#paclitaxel#',
  '#pertuzumab#',
  '#pik3ca#',
  '#pten#',
  '#sgcg#',
  '#sirt1#',
  '#tamoxifen#',
  '#tc2n#',
  '#tp53#',
  '#trastuzumab#',
  'aberr',
  'abil',
  'activ',
  'addit',
  'advanc',
  'affect',
  'age',
  'alpha',
  'also',
  'alter',
  'amplif',
  'analysi',
  'angulo',
  'anti',
  'antibodi',
  'apoptosi',
  'approach',
  'associ',
  'aurora',
  'author',
  'base',
  'beta',
  'bind',
  'biol',
  'biolog',
  'black',
  'breast',
  'cancer',
  'carcinoma',
  'case',
  'cell',
  'certain',
  'chang',
  'chemotherapi',
  'cic',
  'clin',
  'clinic',
  'combin',
  'compar',
  'complex',
  'construct',
  'contain',
  'cont

In [148]:
vocab_matrix=data_frame_creator.open_pickle('vocab_matrix.pickle')

In [149]:
vocab_matrix

<2893x406 sparse matrix of type '<class 'numpy.int64'>'
	with 2941 stored elements in Compressed Sparse Row format>

In [20]:
'vocab_matrix.pickle'[-7:]

'.pickle'

In [134]:
vocab_columns=data_frame_creator.open_pickle('vocabulary.pickle')

In [147]:
len(vocab_columns)

406

In [173]:
col_idx=sparse_matrix_functions.column_indexing(vocab_columns)

In [174]:
col_idx

{'#abcg2#': 0,
 '#acadl#': 1,
 '#ackr3#': 2,
 '#acpp#': 3,
 '#ado#': 4,
 '#afuresertib#': 5,
 '#agfg1#': 6,
 '#aicda#': 7,
 '#airn#': 8,
 '#akt inhibitor gsk2141795#': 9,
 '#akt inhibitor mk2206#': 10,
 '#akt1#': 11,
 '#alb#': 12,
 '#alisertib#': 13,
 '#alpelisib#': 14,
 '#alpi#': 15,
 '#alyref#': 16,
 '#amh#': 17,
 '#anastrozole#': 18,
 '#anpep#': 19,
 '#ar#': 20,
 '#arc#': 21,
 '#areg#': 22,
 '#arhgef7#': 23,
 '#armc9#': 24,
 '#atg12#': 25,
 '#atm#': 26,
 '#atr#': 27,
 '#bank1#': 28,
 '#bax#': 29,
 '#bbc3#': 30,
 '#bcl2#': 31,
 '#bcs1l#': 32,
 '#bevacizumab#': 33,
 '#bid#': 34,
 '#binimetinib#': 35,
 '#birc5#': 36,
 '#bivm#': 37,
 '#bmi1#': 38,
 '#bmp7#': 39,
 '#bpifa4p#': 40,
 '#braf#': 41,
 '#brca1#': 42,
 '#brca2#': 43,
 '#brd2#': 44,
 '#brip1#': 45,
 '#broxuridine#': 46,
 '#buparlisib#': 47,
 '#c19orf66#': 48,
 '#c1qbp#': 49,
 '#ca1#': 50,
 '#capecitabine#': 51,
 '#carbendazim#': 52,
 '#carboplatin#': 53,
 '#cat#': 54,
 '#ccnd1#': 55,
 '#ccng2#': 56,
 '#cct#': 57,
 '#cd24#': 58,


In [26]:
df =data_frame_creator.from_papers_to_panda(path=path,
                      doc_reader=tokenizer.tokenize_many_docs,
                      tokenizer=tokenizer.tokenize,
                      short_list=True,
                      min_df=0.0000001,
                      mat_out='vocab_matrix',
                      vocab_out='vocabulary')

Data has been processed and saved


In [27]:
len(df)

1428

In [28]:
non_empty_rows = sparse_matrix_functions.non_zero_rows(vocab_matrix)

In [29]:
sentences = [orig_sentences[i] for i in non_empty_rows]

In [30]:
sentences

['Something that moves progressivley First thing on your resume Make it as good as possible Highlight your skills  Show complex skills in simple way Clearly defined purposed Has workflow  Translate very complex research onto very simple application Communication is very important Make sure you can get the data promptly  Project should be safe enough to work on the time frame',
 '13  Deregulation of the EGFR PI3K PTEN Akt mTORC1 pathway in breast cancer: possibilities for therapeutic intervention Nicole M. Davis1, Melissa Sokolosky1, Kristin Stadelman1, Stephen L. Abrams1, Massimo Libra2, Saverio Candido2, Ferdinando Nicoletti2, Jerry Polesel3, Roberta Maestro4, Antonino DAssoro5, Lyudmyla Drobot6, Dariusz Rakus7, Agnieszka Gizak7, Piotr Laidler8, Joanna Duliska-Litewka8, Joerg Basecke9, Sanja Mijatovic10, Danijela Maksimovic-Ivanic10, Giuseppe Montalto11,12, Melchiorre Cervello12, Timothy L. Fitzgerald13, Zoya N. Demidenko14, Alberto M. Martelli15, Lucio Cocco15, Linda S. Steelman1 and

In [31]:
files = [doc_names[i] for i in non_empty_rows]

In [32]:
files

['guidelines.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test.txt',
 'test

In [33]:
vocab_mat_clean=sparse_matrix_functions.remove_zero_rows(vocab_matrix)

In [34]:
vocab_mat_clean

<2024x267 sparse matrix of type '<class 'numpy.int64'>'
	with 12901 stored elements in Compressed Sparse Row format>

In [35]:
print(vocab_mat_clean)

  (0, 107)	1
  (0, 229)	1
  (0, 66)	2
  (0, 254)	1
  (0, 220)	1
  (0, 126)	1
  (1, 220)	1
  (1, 8)	1
  (1, 19)	2
  (1, 20)	1
  (1, 0)	1
  (1, 172)	1
  (1, 188)	1
  (1, 53)	1
  (1, 54)	3
  (1, 246)	1
  (1, 170)	2
  (1, 51)	4
  (1, 63)	1
  (1, 5)	1
  (1, 157)	1
  (1, 245)	1
  (1, 247)	2
  (1, 221)	1
  (1, 174)	1
  :	:
  (2015, 54)	2
  (2016, 220)	1
  (2016, 54)	1
  (2016, 44)	1
  (2017, 53)	1
  (2017, 54)	1
  (2017, 210)	1
  (2017, 180)	1
  (2017, 213)	1
  (2017, 231)	1
  (2017, 100)	2
  (2017, 83)	1
  (2017, 22)	1
  (2018, 54)	1
  (2019, 210)	1
  (2020, 110)	1
  (2020, 212)	2
  (2020, 241)	1
  (2021, 216)	2
  (2021, 110)	1
  (2022, 262)	1
  (2022, 187)	1
  (2023, 220)	1
  (2023, 54)	1
  (2023, 44)	1


In [36]:
vocab_mat_clean.shape

(2024, 267)

In [37]:
b = vocab_mat_clean.nonzero()[0]

In [38]:
c = vocab_mat_clean.nonzero()[1]

In [39]:
rows=[]
for i in b:
    rows.append(i)

In [40]:
len(rows)

12901

In [41]:
cols=[]
for i in b:
    cols.append(i)

In [42]:
len(cols)

12901

In [43]:
d=[]
for i in range(len(cols)):
    d.append((rows[i],cols[i]))

In [44]:
print(vocab_mat_clean.sum())

14643


In [45]:
import sklearn.preprocessing as pp

def cosine_similarities(mat):
    col_normed_mat = pp.normalize(mat.tocsc(), axis=0)
    return col_normed_mat.T * col_normed_mat

In [46]:
a = cosine_similarities(vocab_mat_clean)

/Users/ale/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [47]:
list(vocab_mat_clean.getrow(0).nonzero()[1])

[107, 229, 66, 254, 220, 126]

In [48]:
Ks = vocab_mat_clean
value = Ks.data
column_index = Ks.indices
row_pointers = Ks.indptr

In [49]:
value.shape

(12901,)

In [50]:
column_index

array([107, 229,  66, ..., 220,  54,  44], dtype=int32)

In [51]:
row_pointers

array([    0,     6,    30, ..., 12896, 12898, 12901], dtype=int32)

In [52]:
len(column_index)

12901

In [53]:
len(row_pointers)

2025

In [54]:
rows, cols = vocab_matrix.nonzero()

In [55]:
rows

array([   0,    0,    0, ..., 2892, 2892, 2892], dtype=int32)

In [56]:
cols

array([126, 220, 254, ...,  44,  54, 220], dtype=int32)

In [57]:
max(cols)

266

In [136]:
resist_mat = vocab_matrix.multiply(resist_col)

NameError: name 'resist_col' is not defined

In [59]:
sensit_mat = vocab_matrix.multiply(sensit_col)

NameError: name 'sensit_col' is not defined

In [129]:
#gene_dict.values()

In [150]:
vocab_columns.index()

TypeError: index() takes at least 1 argument (0 given)

In [151]:
gene_columns_index =[vocab_columns.index(i) for i in vocab_columns if i in gene_values] 

In [152]:
gene_columns_name =[i for i in vocab_columns if i in gene_values] 

In [153]:
drug_columns_name =[i for i in vocab_columns if i in drug_values] 

In [154]:
drug_columns_index =[vocab_columns.index(i) for i in vocab_columns if i in drug_values] 

In [155]:
vocab_matrix.shape

(2893, 406)

In [156]:
drug_mat=vocab_matrix[:,drug_columns_index]

In [157]:
gene_mat=vocab_matrix[:,gene_columns_index]

In [158]:
gene_mat.T*drug_mat

<328x76 sparse matrix of type '<class 'numpy.int64'>'
	with 536 stored elements in Compressed Sparse Column format>

In [159]:
def drop_columns(X,index_to_drop):
    to_keep = list(set(range(X.shape[1]-1))-set(index_to_drop))    
    new_X = X[:,to_keep]
    return new_X

In [160]:
resist = drop_columns(vocab_matrix, [vocab_columns.index('sensit')])

In [161]:
resist.shape

(2893, 405)

In [162]:
sensit = drop_columns(vocab_matrix, [vocab_columns.index('resist')])

In [163]:
#sparse_matrix_functions.column_indexing(drug_columns_name)

In [164]:
def make_network_matrix(X, vocabulary):
    #drug_columns_name =[i for i in vocabulary if i in drug_values] 
    drug_columns_index =[vocabulary.index(i) for i in vocabulary if i in drug_values]    
    gene_columns_index =[vocabulary.index(i) for i in vocabulary if i in gene_values] 
    #gene_columns_name =[i for i in vocabulary if i in gene_values] 
    gene_mat=X[:,gene_columns_index]
    drug_mat=X[:,drug_columns_index]
    network_matrix = gene_mat.T*drug_mat
    return network_matrix

In [165]:
def get_network_rows(vocabulary):
    genes = {}
    genes_columns_names=[i for i in vocabulary if i in gene_values]
    for idx,name in enumerate(genes_columns_names):
        genes[idx]=name
    return genes

In [166]:
network_genes=get_network_rows(vocab_columns)

In [167]:
def get_network_columns(vocabulary):
    drugs = {}
    drugs_columns_names=[i for i in vocabulary if i in drug_values]
    for idx,name in enumerate(drugs_columns_names):
        drugs[idx]=name
    return drugs

In [168]:
network_drugs =get_network_columns(vocab_columns)

In [180]:
network_drugs[75]

'#voxtalisib#'

In [181]:
network_genes[75]

'#dgcr2#'

In [182]:
col_idx['#dgcr2#']

97

In [183]:
col_idx[network_drugs[75]]

396

In [184]:
sensit_network_matrix = make_network_matrix(sensit, vocab_columns)

In [185]:
#sensit_network_matrix.nonzero()

In [186]:
sensit_network_matrix

<328x76 sparse matrix of type '<class 'numpy.int64'>'
	with 536 stored elements in Compressed Sparse Column format>

In [187]:
len(list(sensit_network_matrix.nonzero()[1]))

536

In [188]:
a =zip(list(sensit_network_matrix.nonzero()[0]),list(sensit_network_matrix.nonzero()[1]))

In [189]:

for i in a:
    pairs = (network_genes[i[0]], network_drugs[i[1]])
    r =np.zeros((sensit_network_matrix.shape[0],1))
    r[i[0]]=1
    c =np.zeros((sensit_network_matrix.shape[1],1))
    c[i[1]]=1
    counts =(r.T).dot(sensit_network_matrix.dot(c))
    print(pairs)
    print(counts)

('#abcg2#', '#mitoxantrone#')
[[ 1.]]
('#acpp#', '#binimetinib#')
[[ 6.]]
('#acpp#', '#buparlisib#')
[[ 6.]]
('#acpp#', '#dalotuzumab#')
[[ 3.]]
('#acpp#', '#everolimus#')
[[ 6.]]
('#acpp#', '#pi3k/mtor inhibitor bez235#')
[[ 6.]]
('#acpp#', '#ridaforolimus#')
[[ 18.]]
('#acpp#', '#temsirolimus#')
[[ 9.]]
('#acpp#', '#vinorelbine#')
[[ 3.]]
('#acpp#', '#voxtalisib#')
[[ 6.]]
('#ado#', '#trastuzumab emtansine#')
[[ 2.]]
('#ado#', '#trastuzumab#')
[[ 2.]]
('#akt1#', '#afuresertib#')
[[ 1.]]
('#akt1#', '#akt inhibitor gsk2141795#')
[[ 10.]]
('#akt1#', '#akt inhibitor mk2206#')
[[ 17.]]
('#akt1#', '#bevacizumab#')
[[ 2.]]
('#akt1#', '#binimetinib#')
[[ 1.]]
('#akt1#', '#buparlisib#')
[[ 14.]]
('#akt1#', '#capecitabine#')
[[ 2.]]
('#akt1#', '#carboplatin#')
[[ 5.]]
('#akt1#', '#cisplatin#')
[[ 7.]]
('#akt1#', '#cyclophosphamide#')
[[ 2.]]
('#akt1#', '#docetaxel#')
[[ 2.]]
('#akt1#', '#doxorubicin#')
[[ 1.]]
('#akt1#', '#everolimus#')
[[ 120.]]
('#akt1#', '#exemestane#')
[[ 7.]]
('#akt1#', '

('#glys1#', '#vinorelbine#')
[[ 4.]]
('#glys1#', '#voxtalisib#')
[[ 6.]]
('#hdac9#', '#tamoxifen citrate#')
[[ 2.]]
('#hpp1#', '#anastrozole#')
[[ 1.]]
('#hpp1#', '#bevacizumab#')
[[ 1.]]
('#hpp1#', '#capecitabine#')
[[ 1.]]
('#hpp1#', '#carbendazim#')
[[ 1.]]
('#hpp1#', '#carboplatin#')
[[ 1.]]
('#hpp1#', '#cisplatin#')
[[ 3.]]
('#hpp1#', '#cyclophosphamide#')
[[ 1.]]
('#hpp1#', '#epirubicin#')
[[ 1.]]
('#hpp1#', '#everolimus#')
[[ 12.]]
('#hpp1#', '#exemestane#')
[[ 1.]]
('#hpp1#', '#letrozole#')
[[ 1.]]
('#hpp1#', '#paclitaxel#')
[[ 7.]]
('#hpp1#', '#trastuzumab#')
[[ 1.]]
('#hpp1#', '#vinorelbine#')
[[ 1.]]
('#hr#', '#akt inhibitor gsk2141795#')
[[ 2.]]
('#hr#', '#akt inhibitor mk2206#')
[[ 3.]]
('#hr#', '#buparlisib#')
[[ 2.]]
('#hr#', '#carboplatin#')
[[ 1.]]
('#hr#', '#cisplatin#')
[[ 1.]]
('#hr#', '#everolimus#')
[[ 21.]]
('#hr#', '#exemestane#')
[[ 1.]]
('#hr#', '#mtor kinase inhibitor cc-223#')
[[ 1.]]
('#hr#', '#paclitaxel#')
[[ 1.]]
('#hr#', '#pi3k/mtor inhibitor bez235#')


[[ 1.]]
('#tnip1#', '#trastuzumab#')
[[ 1.]]
('#tp53#', '#cyclophosphamide#')
[[ 2.]]
('#tp53#', '#docetaxel#')
[[ 1.]]
('#tp53#', '#doxorubicin#')
[[ 2.]]
('#tp53#', '#epirubicin#')
[[ 1.]]
('#tp53#', '#fluorouracil#')
[[ 1.]]
('#tp53#', '#hydroxyurea#')
[[ 1.]]
('#tp53#', '#iproplatin#')
[[ 2.]]
('#tp53#', '#paclitaxel#')
[[ 1.]]


In [190]:
def extract_gene_drug_pairs(matrix):
    a =zip(list(matrix.nonzero()[0]),list(matrix.nonzero()[1])) 
    pairs = []
    counts = []
    for pair in a:
        gendrug = (network_genes[pair[0]], network_drugs[pair[1]])
        r =np.zeros((matrix.shape[0],1))
        r[pair[0]]=1
        c =np.zeros((matrix.shape[1],1))
        c[pair[1]]=1
        count =(r.T).dot(matrix.dot(c))
        pairs.append(gendrug)
        counts.append(count)
    return (pairs, counts)
    

In [191]:
pairs, counts = extract_gene_drug_pairs(sensit_network_matrix)

In [192]:
pairs

[('#abcg2#', '#mitoxantrone#'),
 ('#acpp#', '#binimetinib#'),
 ('#acpp#', '#buparlisib#'),
 ('#acpp#', '#dalotuzumab#'),
 ('#acpp#', '#everolimus#'),
 ('#acpp#', '#pi3k/mtor inhibitor bez235#'),
 ('#acpp#', '#ridaforolimus#'),
 ('#acpp#', '#temsirolimus#'),
 ('#acpp#', '#vinorelbine#'),
 ('#acpp#', '#voxtalisib#'),
 ('#ado#', '#trastuzumab emtansine#'),
 ('#ado#', '#trastuzumab#'),
 ('#akt1#', '#afuresertib#'),
 ('#akt1#', '#akt inhibitor gsk2141795#'),
 ('#akt1#', '#akt inhibitor mk2206#'),
 ('#akt1#', '#bevacizumab#'),
 ('#akt1#', '#binimetinib#'),
 ('#akt1#', '#buparlisib#'),
 ('#akt1#', '#capecitabine#'),
 ('#akt1#', '#carboplatin#'),
 ('#akt1#', '#cisplatin#'),
 ('#akt1#', '#cyclophosphamide#'),
 ('#akt1#', '#docetaxel#'),
 ('#akt1#', '#doxorubicin#'),
 ('#akt1#', '#everolimus#'),
 ('#akt1#', '#exemestane#'),
 ('#akt1#', '#gemcitabine#'),
 ('#akt1#', '#lapatinib#'),
 ('#akt1#', '#letrozole#'),
 ('#akt1#', '#mtor kinase inhibitor cc-223#'),
 ('#akt1#', '#paclitaxel albumin-stabiliz

In [96]:
import networkx as nx

In [193]:
def back_to_original_indeces(pairs):
    original_index = []
    for item in pairs:
        original = (col_idx[item[0]], col_idx[item[1]])
        original_index.append(original)
    return pairs ,original_index
    

In [194]:
def back_to_original_index(pair):
    original = (col_idx[pair[0]],col_idx[pair[1]])
    return pair,original

In [195]:
back_to_original_index(('#abcg2#', '#mitoxantrone#'))

(('#abcg2#', '#mitoxantrone#'), (0, 221))

In [229]:
def back_to_original_indeces(pairs):
    original_indeces = {}
    for item in pairs:
        original = back_to_original_index(item)
        original_indeces[original[0]]=original[1]
    return original_indeces

In [230]:
original_indeces = back_to_original_indeces(pairs)

In [231]:
original_indeces

{('#abcg2#', '#mitoxantrone#'): (0, 221),
 ('#acpp#', '#binimetinib#'): (3, 35),
 ('#acpp#', '#buparlisib#'): (3, 47),
 ('#acpp#', '#dalotuzumab#'): (3, 92),
 ('#acpp#', '#everolimus#'): (3, 127),
 ('#acpp#', '#pi3k/mtor inhibitor bez235#'): (3, 276),
 ('#acpp#', '#ridaforolimus#'): (3, 316),
 ('#acpp#', '#temsirolimus#'): (3, 363),
 ('#acpp#', '#vinorelbine#'): (3, 395),
 ('#acpp#', '#voxtalisib#'): (3, 396),
 ('#ado#', '#trastuzumab emtansine#'): (4, 385),
 ('#ado#', '#trastuzumab#'): (4, 386),
 ('#akt1#', '#afuresertib#'): (11, 5),
 ('#akt1#', '#akt inhibitor gsk2141795#'): (11, 9),
 ('#akt1#', '#akt inhibitor mk2206#'): (11, 10),
 ('#akt1#', '#bevacizumab#'): (11, 33),
 ('#akt1#', '#binimetinib#'): (11, 35),
 ('#akt1#', '#buparlisib#'): (11, 47),
 ('#akt1#', '#capecitabine#'): (11, 51),
 ('#akt1#', '#carboplatin#'): (11, 53),
 ('#akt1#', '#cisplatin#'): (11, 74),
 ('#akt1#', '#cyclophosphamide#'): (11, 85),
 ('#akt1#', '#docetaxel#'): (11, 100),
 ('#akt1#', '#doxorubicin#'): (11, 1

In [198]:
a =np.zeros((vocab_matrix.shape[1],1))

In [199]:
a[0]=1
a[221]=1

In [200]:
a.shape

(406, 1)

In [202]:
b = scs.lil_matrix((vocab_matrix.shape[1],1))

In [203]:
b

<406x1 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in LInked List format>

In [204]:
b[list((0,221))]=1

In [205]:
print(b)

  (0, 0)	1.0
  (221, 0)	1.0


In [206]:
vocab_matrix*b

<2893x1 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [207]:
vocab_matrix.data

array([1, 2, 1, ..., 1, 1, 1])

In [208]:
vocab_test =vocab_matrix

In [209]:
vocab_test[vocab_test>1]=1

In [210]:
vocab_test>1

<2893x406 sparse matrix of type '<class 'numpy.bool_'>'
	with 0 stored elements in Compressed Sparse Row format>

In [211]:
(vocab_test*b>1).nonzero()[0]

array([448], dtype=int32)

In [212]:
orig_sentences[448]

'In the studies with the side populations from MCF-7 and MDA-MB-231 cells, increased ABCG2 was observed in the side populations as well as elevated resistance to the chemotherapeutic drug mitoxantrone.'

In [528]:
def get_evidence_sentences(gene, drug, max_number):
    gen = '#'+gene+'#'
    drg = '#'+drug+'#'
    gene = gene.lower()
    drug = drug.lower()
    if gen not in gene_values:
        try:
            gen=gene_dict[gene]
        except KeyError:
            return ("Gene: {} not in list".format(gene))
    if drg not in drug_values:
        try:
            drg=drug_dict[drug]
        except KeyError:
            return ("Drug: {} not in list".format(drug))
    try:
        indices = original_indeces[(gen, drg)]
    except KeyError:
        return('There is no evidence of interaction'
        ' between the gene {} and the drug {}'. format(gen[1:-1],drg[1:-1]))
    test_col = scs.lil_matrix((vocab_matrix.shape[1],1))
    test_col[list(indices)]=1
    index_evidence = (vocab_matrix*test_col>1).nonzero()[0]
    evidence_list = []
    #gene_key = [k for k, v in gene_dict.items() if v == gen]
    #drug_key = [k for k, v in drug_dict.items() if v == drg]
    while len(evidence_list) < max_number and len(evidence_list) < len(index_evidence) :
        for index in index_evidence:
            sent = orig_sentences[index]#.lower()
            #gene_evd = [word for word in sent.replace(',','').replace('.','').replace(':','').split(' ') if word in gene_key][0]
            #drug_evd = [word for word in sent.replace(',','').replace('.','').replace(':','').split(' ') if word in drug_key][0]
            if len(sent)<500:
                evidence_list.append('{}: {}'.format(doc_names[index] ,sent))
            else:
                evidence_list.append('{}: sentence too long to display'.format(doc_names[index]))  
    return evidence_list
        
    

In [529]:
doc_names[448]

'test.txt'

In [530]:
get_evidence_sentences('abcg2','mitoxantrone',10)

['test.txt: In the studies with the side populations from MCF-7 and MDA-MB-231 cells, increased ABCG2 was observed in the side populations as well as elevated resistance to the chemotherapeutic drug mitoxantrone.']

In [531]:
get_evidence_sentences('akt1','lapatinib',2)

['test.txt: In BT474 and MCF10A breast cancer cells transfected with the construct encoding HER2-T798M mutation, elevated HER2 kinase activity was detected and lapatinib did not block phosphorylation of HER2, EGFR3 or downstream Akt and ERK1 2.',
 'test.txt: Treatment of the lapatinib-resistant HER2+ cells with Src inhibitors suppressed the PI3K PTEN Akt mTOR pathway as well as  Figure 5: Sites of Targeting the EGFR PI3K PTEN Akt mTORC Pathway with Small Molecule Membrane-Permeable Inhibitors and Monoclonal Antibodies (MoAbs).',
 'test.txt: sentence too long to display']

In [535]:
almost_vocab = data_frame_creator.open_pickle('almost_vocab_columns.pickle')

In [537]:
with open('almost_vocab.txt','w') as f:
    f.write("\n".join(almost_vocab))

In [541]:
almost_vocab_drugs_index =[almost_vocab.index(i) for i in almost_vocab if i in drug_values] 

In [588]:
almost_vocab_drug_name =[i for i in almost_vocab if i in drug_values] 

In [589]:
with open('almost_vocab_drugs.txt','w') as f:
    f.write("\n".join(almost_vocab_drug_name))

In [590]:
repeat_names =[]
for drug in almost_vocab_drug_name:
    for drug2 in almost_vocab_drug_name:
        if drug2[1:-1] in drug[1:-1] and len(drug2)<len(drug):
            repeat_names.append((drug2,drug))
            

In [591]:
len(repeat_names)

124

In [592]:
repeat_names[0][1]

'#7-ethyl-10-hydroxycamptothecin#'

In [593]:
drug_key = [k for k, v in drug_dict.items() if v == repeat_names[0][1]]

In [594]:
drug_key

['7-ethyl-10-hydroxycamptothecin',
 '7-ethyl-10-hydroxy-20(s)-camptothecin',
 'sn 38',
 'sn-38']

In [607]:
for repeat in repeat_names:
    drug_dict[repeat[1][1:-1]] = repeat[0]
    drug_key = [k for k, v in drug_dict.items() if v == repeat[1]]
    for drug in drug_key:
        #print(drug,repeat[0])
        drug_dict[drug]=repeat[0]
        

In [608]:
repeat_names_2 =[]
for drug in almost_vocab_drug_name_2:
    for drug2 in almost_vocab_drug_name_2:
        if drug2[1:-1] in drug[1:-1] and len(drug2)<len(drug):
            repeat_names_2.append((drug2,drug))

In [609]:
repeat_names_2

[]

In [616]:
drug_dict['acridine carboxamide']

'#acridine#'

In [627]:
with open('drug_dictionary_final.pickle', 'wb') as handle:
    pickle.dump(drug_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [629]:
test_dict = data_frame_creator.open_pickle('drug_dictionary_final.pickle')

In [630]:
test_dict['acridine carboxamide']

'#acridine#'

In [631]:
with open('drug_dictionary_final.json', 'w') as fp:
    json.dump(drug_dict, fp, sort_keys=True, indent=4)